In [1]:
import sys
import deepwalk
import numpy as np

In [2]:
def check_method(method):
    val = None
    if method == "kmedoids" or method =="Kmedoids":
        val = "kmedoids"
    elif method =="kmeans" or method == "Kmeans":
        val = "kmeans"
    else:
        sys.exit(f"methodの値が無効です method : {method}")
    return val

In [3]:
def check_data(data):
    if data=="football" or data == "polbooks" or data=="karateclub":
        pass
    else:
    
        sys.exit(f"変数dataの値が無効です data : {data}")

In [4]:
np.random.choice(3,size=None,)

2

In [5]:
G = deepwalk.generate_Graph("karateclub")
d = dict(G.edges)

In [6]:
dg = deepwalk.from_networkx(G)

In [8]:
#実行回数(埋め込み-クラスタリング-ARI)
TIME = 100

"""
data : 使用するデータ
football   : "football"
polbooks   : "polbooks"
karateclub : "karateclub"
"""
data = "karateclub"
check_data(data)

# インプットするファイル名
INPUT = deepwalk.generate_Graph(data)

#正解ラベル
TRUE_LABEL = deepwalk.get_label_list(data)

#インプットするグラフが無向グラフかどうか(True:無向グラフ,　False:有向グラフ)
UNDIRECTED = True

# １ノードあたりの実行数
NUMBER_WALKS =10

# 1ウォークあたりの長さ
WALK_LENGTH = 20

# ランダムシード値
SEED = 0

# 埋め込み後の各ノードの次元数
REPRESENTATION_SIZE = 2

# skipgramので使われるウィンドウサイズ
WINDOW_SIZE = 4

# 並列プロセス数
WORKERS = 1

#クラスタ数
N_CLUSTER = 2

#クラスタリング手法
"""
kmedoids : "kmedoids"
kmeans   : "kmeans"
"""
METHOD = check_method("kmedoids")

#結果を表示するかどうか
SHOW = True


"""
以下はパラメータを変更して連続で実行する際に使用
(for文でリストを与えて実行)
"""

# １ノードあたりの実行数
NUMBER_WALKS_LIST = [1, 2, 4, 6, 8, 10, 15, 20, 25, 30]

# 1ウォークあたりの長さ
WALK_LENGTH_LIST = [5]

# ウィンドウサイズリスト
WINDOW_SIZE_LIST = [1, 2, 3, 4, 5,6,7,8,9,10]


In [9]:
"""
    グラフ生成
    G = from_networkx(INPUT:networkxグラフインスタンス, 無効グラフかどうか:boolean)
    """
G = deepwalk.from_networkx(INPUT, True)
# ノード数出力
print("Number of nodes: {}".format(len(G.nodes())))

# ウォーク数算出(ノード数*1ノードあたりのウォーク数)と出力
num_walks = len(G.nodes()) * NUMBER_WALKS
print("Number of walks: {}".format(num_walks))

# データサイズ(ウォーク数*ウォークの長さ)の算出と出力
data_size = num_walks * WALK_LENGTH
print("Data size (walks*length): {}".format(data_size))



Number of nodes: 34
Number of walks: 340
Data size (walks*length): 6800


In [10]:
ARI_list, max_walks, min_walks, max_vec, min_vec, max_pred, min_pred = deepwalk.multi_exection(
        TIME, INPUT, UNDIRECTED, NUMBER_WALKS, WALK_LENGTH, REPRESENTATION_SIZE, WINDOW_SIZE, WORKERS, N_CLUSTER, METHOD, TRUE_LABEL, SHOW)


------------------------1回目実行-----------------------------
ari : 0.8822575413558222
------------------------2回目実行-----------------------------
ari : 0.8822575413558222
------------------------3回目実行-----------------------------
ari : 0.8822575413558222
------------------------4回目実行-----------------------------
ari : 0.8822575413558222
------------------------5回目実行-----------------------------
ari : 0.6684671059738576
------------------------6回目実行-----------------------------
ari : 0.8822575413558222
------------------------7回目実行-----------------------------
ari : 0.8822575413558222
------------------------8回目実行-----------------------------
ari : 0.8822575413558222
------------------------9回目実行-----------------------------
ari : 0.8822575413558222
------------------------10回目実行-----------------------------
ari : 0.8822575413558222
------------------------11回目実行-----------------------------
ari : 0.8822575413558222
------------------------12回目実行-----------------------------
ari : 0.882257

In [18]:
#maxvecをcsvに出力
with open(f"max_vec_{data}.csv", "w") as f:
    for i in range(len(max_vec)):
        f.write(f"{max_vec[i][0]},{max_vec[i][1]}\n")
